In [1]:
from imports.ASLDataset import ASLDataset
import torch
#torch.set_default_dtype(torch.float32) 

l = ASLDataset(letters=['a', 'b'], overwrite_processing=False)

Processing...


processing:   0%|          | 0/8400 [00:00<?, ?it/s]

Done!


In [2]:
from torch_geometric.loader import DataLoader

dl = DataLoader(l, 3, shuffle=True)

In [4]:
import os.path as osp
import numpy as np
import math
import shutil
import os

import torch
from torch.utils.data.dataset import Subset
import torch.nn as nn
from torch import autograd
from torch.autograd import Variable
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.utils import normalized_cut
from torch_geometric.nn import SplineConv, voxel_grid, max_pool, max_pool_x, graclus




transform = T.Cartesian(cat=False)



class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SplineConv(1, 64, dim=3, kernel_size=5)
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.conv2 = SplineConv(64, 128, dim=3, kernel_size=5)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.conv3 = SplineConv(128, 256, dim=3, kernel_size=5)
        self.bn3 = torch.nn.BatchNorm1d(256)
        self.conv4 = SplineConv(256, 512, dim=3, kernel_size=5)
        self.bn4 = torch.nn.BatchNorm1d(512)
        self.fc1 = torch.nn.Linear(64 * 512, 1024)
        self.fc2 = torch.nn.Linear(1024, 2)

    def forward(self, data):
        print(f'min pos {torch.min(data.pos)}, max pos {torch.max(data.pos)}')
        inp = self.conv1(data.x, data.edge_index, data.edge_attr)
        data.x = F.elu(inp)
        data.x = self.bn1(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=[50,50])
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        
        data.x = F.elu(self.conv2(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn2(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=[50,50])
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        data.x = F.elu(self.conv3(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn3(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=[70,70])
        data = max_pool(cluster, data, transform=T.Cartesian(cat=False))

        data.x = F.elu(self.conv4(data.x, data.edge_index, data.edge_attr))
        data.x = self.bn4(data.x)
        cluster = voxel_grid(data.pos, batch=data.batch, size=[100,100])
        print('max clus', torch.max(cluster))
        x, _ = max_pool_x(cluster, data.x, data.batch, size=64)

        x = x.view(-1, self.fc1.weight.size(1))
        x = F.elu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)


def train(epoch, train_loader):
    model.train()

    if epoch == 60:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.0001

    if epoch == 110:
        for param_group in optimizer.param_groups:
            param_group['lr'] = 0.00001

    for i, data in enumerate(tqdm(train_loader, desc='batches')):
        with autograd.detect_anomaly():
            data = data.to(device)
            #print(data.y)
            optimizer.zero_grad()
            end_point = model(data)
            
            loss = F.nll_loss(end_point, data.y)
            pred = end_point.max(1)[1]
            acc = (pred.eq(data.y).sum().item())/len(data.y)
            
            loss.backward()
            optimizer.step()
            
            if i % 10 == 0:
                print({'epoch': epoch,'batch': i + 1,'loss': loss.item(),'acc': acc})

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
print(device)
#model = torch.load('./runs_model/model.pkl')
#model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

cuda


In [6]:
model.train()

Net(
  (conv1): SplineConv(1, 64, dim=3)
  (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): SplineConv(64, 128, dim=3)
  (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): SplineConv(128, 256, dim=3)
  (bn3): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): SplineConv(256, 512, dim=3)
  (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=32768, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=2, bias=True)
)

In [ ]:
from tqdm.auto import tqdm


for epoch in range(1, 2):
    
    #train_data_aug = T.Compose([T.Cartesian(cat=False), T.RandomFlip(axis=0, p=0.3), T.RandomScale([0.95,0.999]), T.RandomFlip(axis=1, p=0.2)])
    train_loader = dl
    
    train(epoch, train_loader)

batches:   0%|          | 0/2800 [00:00<?, ?it/s]

/tmp/ipykernel_22715/312222590.py:84: UserWarning: Anomaly Detection has been enabled. This mode will increase the runtime and should only be enabled for debugging.
  with autograd.detect_anomaly():


min pos 0.0, max pos 258.0379943847656
max clus tensor(46, device='cuda:0')
{'epoch': 1, 'batch': 1, 'loss': 0.6376842260360718, 'acc': 1.0}
min pos 0.0, max pos 328.3900146484375
max clus tensor(52, device='cuda:0')
min pos 0.0, max pos 236.03199768066406
max clus tensor(46, device='cuda:0')
min pos 0.0, max pos 328.5820007324219
max clus tensor(71, device='cuda:0')
min pos 0.0, max pos 328.4440002441406
max clus tensor(59, device='cuda:0')
min pos 0.0, max pos 238.0
max clus tensor(32, device='cuda:0')
min pos 0.0, max pos 327.37200927734375
max clus tensor(57, device='cuda:0')
min pos 0.0, max pos 305.7720031738281
max clus tensor(66, device='cuda:0')
min pos 0.0, max pos 238.0
max clus tensor(31, device='cuda:0')
min pos 0.0, max pos 311.6700134277344
max clus tensor(48, device='cuda:0')
min pos 0.0, max pos 328.51800537109375
max clus tensor(68, device='cuda:0')
{'epoch': 1, 'batch': 11, 'loss': 0.24112719297409058, 'acc': 0.6666666666666666}
min pos 0.0, max pos 235.6620025634765

In [ ]:
import torch
model = torch.load('model.pkl')
model

In [ ]:
import pandas as pd

log = pd.read_csv('log', names = ['epoch', 'batch', 'loss', 'acc'])
log

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


plt.plot(log['loss'])

In [ ]:
sns.lineplot(x=log.index, y=log['loss'], )

In [ ]:
from imports.NVSModel import Net

model = Net(
    n_classes=2
    #layer_sizes=[64, 128],
    #voxel_sizes=[4, 6]
)
model

In [ ]:
model.fc1

In [ ]:
512*64